In [22]:
import networkx as nx
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from docplex.mp.model import Model

from qc_grader.graph_util import display_maxcut_widget, QAOA_widget, graphs
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit_optimization import QuadraticProgram
from typing import List
import plotly.graph_objects as go
import matplotlib as mpl
import pandas as pd
from IPython.display import clear_output
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
from qiskit import Aer
from qiskit import QuantumCircuit
from qiskit.visualization import plot_state_city
from qiskit.algorithms.optimizers import COBYLA, SLSQP, ADAM
from qiskit.algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from time import time
from copy import copy
from qc_grader.graph_util import display_maxcut_widget, QAOA_widget, graphs
import math
from qiskit.visualization import plot_histogram
import operator
from collections import Counter



matplotlib.rcParams['figure.dpi'] = 100

### QAOA on QUBO problems

In [23]:
def get_qoao_result(quadratic_program, simulator='statevector_simulator'):
    backend = Aer.get_backend(simulator)
    qaoa = QAOA(optimizer = ADAM(), quantum_instance = backend, reps=1, initial_point = [0.1,0.1])
    eigen_optimizer = MinimumEigenOptimizer(min_eigen_solver = qaoa)
#     quadratic_program = quadratic_program_from_graph(G)
    print("succesfully created a quadratic program")
    start_time = time()
    result = eigen_optimizer.solve(quadratic_program)
    end_time = time()
    print("Time taken: ", end_time - start_time)
    print(result)
    return result

In [30]:
# construct model using docplex
mdl = Model('ex6')

s11 = mdl.binary_var(name='s11')
s12 = mdl.binary_var(name='s12')
s21 = mdl.binary_var(name='s21')
s22 = mdl.binary_var(name='s22')
# u = mdl.continuous_var(name='u')

# mdl.minimize(4*s11 + 5*s12 + 6*s21 + 5 * (u-2)**2)
mdl.minimize(4*s11 + 5*s12 + 6*s21 + 4*s22)
# mdl.minimize(4*s11 + 5*s12 + 6*s21 + 4*s22 + 3*(s11)**2 + 2*(s12)**2 + (s21)**2 + 2*(s22)**2)
mdl.add_constraint(s11 + s12 + s21 + s22 == 1, "cons1")
# mdl.add_constraint(v + w + t >= 1, "cons2")
# mdl.add_constraint(v + w == 1, "cons3")

# load quadratic program from docplex model
qp = QuadraticProgram()
qp.from_docplex(mdl)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: ex6

Minimize
 obj: 4 s11 + 5 s12 + 6 s21 + 4 s22
Subject To
 cons1: s11 + s12 + s21 + s22 = 1

Bounds
 0 <= s11 <= 1
 0 <= s12 <= 1
 0 <= s21 <= 1
 0 <= s22 <= 1

Binaries
 s11 s12 s21 s22
End



In [31]:
get_qoao_result(qp)

succesfully created a quadratic program
Time taken:  1.665144681930542
optimal function value: 4.0
optimal value: [1. 0. 0. 0.]
status: SUCCESS


optimal function value: 4.0
optimal value: [1. 0. 0. 0.]
status: SUCCESS